# PyJive workshop: stability of a house-shaped frame

## Case definition
In this workshop, you are asked to set up your own problem and analyse it from different angles, using the `FrameModel` and different modules from pyJive as discussed in previous workshops. The case that is studied is the one illustrated below. 

<center><img src="https://raw.githubusercontent.com/fmeer/public-files/main/ciem5110/formativeFrame.png" alt="Simple frame" width="300"/></center>



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import os
import sys

pyjivepath = '../../../pyjive/'
sys.path.append(pyjivepath)

if not os.path.isfile(pyjivepath + 'utils/proputils.py'):
    print('\n\n**pyjive cannot be found, adapt "pyjivepath" above or move notebook to appropriate folder**\n\n')
    raise Exception('pyjive not found')

from utils import proputils as pu
import main
from names import GlobNames as gn

%matplotlib widget

## Linear elastic analysis
Define your own pair of geometry file and input file and perform a linear elastic analysis. A good starting point could be the `frameNonlin.pro` input file from the workshop on buckling. Make sure you switch off nonlinearity. 

In [ ]:
# In the input file, the SolverModule is specified
# for a linear-elastic problem, we only need to solve a system of equations once.
# the load level is adapted to be high enough for meaningful comparison later on
# In the solution the moment line is visualized on the deformed frame
# no slider because there are no "time steps"

props = pu.parse_file('house-linear.pro')
globdat = main.jive(props)

#### Storing results
Below, a function is defined to looks up load-displacement data from globdat. The function is then called to store the data from the linear elastic analysis. This can later be used to compare the results from different analyses in a single diagram.

You need to modify the function to get relevant load/displacement data. Note that in order to record data for a particular node group, you need to specify this group in the `loaddisp` part of the `.pro`-file. 



In [ ]:
def getFu(globdat):
    F = abs(globdat['loaddisp']['top']['load']['dy'])
    u = abs(globdat['loaddisp']['top']['disp']['dy'])
    return np.vstack((u,F))

lin_elas = getFu(globdat)

# pad with zeros because linear elastic gives only one force-displacement point
lin_elas = np.hstack((np.zeros((2,1)), lin_elas))

## Linear buckling analysis
Now perform linear buckling analysis. You are recommended to make a new input file because quite several modifications need to be made. Again, you can take one from the buckling workshop as starting point. 

In [ ]:
# In this input file the LinBuckModule is used instead of the SolverModule
# Load magnitude is set to 1, such that scale factor is directly interpretable as buckling load

# This time the plot does have a slider the solution contains multiple buckling modes

props = pu.parse_file('house-lb.pro')
globdat = main.jive(props)
bucklingLoad = globdat[gn.LBFACTORS][0]

## Geometrically nonlinear elastic analysis
The next step is to perform geometrically nonlinear elastic analysis. You can take the input file from the linear-elastic analysis and overwrite one of the entries in the notebook. How do the results compare to the results from linear buckling analysis (in terms of buckling load and buckling mode)?

In [ ]:
# Again with a new input file, changes with respect to house-linear.pro are 
# - subtype = "nonlin" (to use a nonlinear solver)
# - NonlinModule instead of SolverModule to use incremental-iterative scheme
# - displacement control instead of load control
# - FrameViewModule is interactive: there will be a slider this time
# Arclength-control is not necessary here, but also works


props = pu.parse_file('house-nonlin.pro')
globdat = main.jive(props)

In [ ]:
# A first comparison between load-displacement results from two analyses

nonlin_elas = getFu(globdat)


plt.close('all')
plt.plot(nonlin_elas[0],nonlin_elas[1],label='nonlinear')
plt.plot(lin_elas[0],lin_elas[1],label='linear')
plt.plot([0,0.6],[bucklingLoad,bucklingLoad],'--',label='linear buckling analysis')
plt.legend()
plt.xlabel('displacement [m]')
plt.ylabel('force [N]')
plt.show()



## Geometrically linear elastic/plastic analysis
Next, perform a geometrically linear analysis with plastic hinges. Again, take the previous input file as starting point.

In [ ]:
# Two inputs are added related to plasticity
# For geometric linear analysis, set subtype to 'linear'

props['model']['frame']['plastic'] = 'True'
props['model']['frame']['Mp'] = 3000;
props['model']['frame']['subtype'] = 'linear'

globdat = main.jive(props)

lin_plas = getFu(globdat)

## Geometrically nonlinear elastic/plastic analysis
As last analysis, perform a complete nonlinear finite element simulation with geometric nonlinearity and plastic hinges. 

In [ ]:
# Make element formulation geometrically nonlinear again
props['model']['frame']['subtype'] = 'nonlin'
globdat = main.jive(props)

nonlin_plas = getFu(globdat)


### Comparison with results
Finally, compare the results from different analysis. 
Also include the rigid-plastic 2nd order analysis result (see [pdf](https://gitlab.tudelft.nl/cm/public/drive/-/raw/main/frame-assignment/CIEM5110_U1_assignmentA_solution.pdf) for background):

$$ 
F \approx \frac{6M_\mathrm{p}}{L}\left(1-2\textstyle\frac13\theta\right)
$$

Suppose you want to check this analytical solution, what can you change to the model inputs to get more definite insight in its validity? 

In [ ]:
# Compare results from the different analyses in a single figure
plt.figure()
plt.plot(lin_elas[0],lin_elas[1],label='linear elastic')
plt.plot(nonlin_elas[0],nonlin_elas[1],label='nonlinear elastic')
plt.plot(lin_plas[0],lin_plas[1],label='linear plastic')
plt.plot(nonlin_plas[0],nonlin_plas[1],label='nonlinear plastic')
plt.xlabel('displacement [m]')
plt.ylabel('force [N]')

# Add analytical solution (note that for this geometry, theta=uy[top])
Mp = props['model']['frame']['Mp']
L = 1
theta = np.arange(0,0.41,0.01)
F_analytical = 6*Mp/L*(1-7/3*theta)
plt.plot(theta,F_analytical,'--',label='2nd order plastic')
plt.legend()


In [ ]:
props['model']['frame']['EI'] = 10.e6
globdat = main.jive(props)

rig_plas = getFu(globdat)
plt.plot(rig_plas[0],rig_plas[1])

In [ ]:
plt.figure()
plt.plot(lin_elas[0],lin_elas[1],label='linear elastic')
plt.plot(nonlin_elas[0],nonlin_elas[1],label='nonlinear elastic')
plt.plot(lin_plas[0],lin_plas[1],label='linear plastic')
plt.plot(nonlin_plas[0],nonlin_plas[1],label='nonlinear plastic')
plt.plot(theta,F_analytical,'--',label='2nd order plastic')
plt.plot(rig_plas[0],rig_plas[1],label='rigid plastic')
plt.xlabel('displacement [m]')
plt.ylabel('force [N]')
plt.legend()

## Bonus: go back to nonlinear elastic and add perturbation
By slightly moving the top node in the original geometry, symmetry of the structure is perturbed and a different solution is revealed. 

**Move the slider to see what happens to the displacements around the bifurcation point. Compare the deformation pattern to the critical buckling mode from linear buckling analysis.**

In [ ]:
props = pu.parse_file('house-nonlin.pro')
props['init']['mesh']['file'] = 'house-imperfect.geom'

globdat = main.jive(props)

In [ ]:
perturbed = getFu(globdat)
plt.figure()
plt.plot(nonlin_elas[0],nonlin_elas[1],label='symmetric')
plt.plot(perturbed[0],perturbed[1],label='perturbed')
plt.plot([0,0.6],[bucklingLoad,bucklingLoad],'--',label='linear buckling analysis')
plt.xlabel('displacement [m]')
plt.ylabel('force [N]')
plt.legend()